In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [5]:
data = pd.read_csv("ICDF_Distribution.dat",sep = " ")

In [7]:
data.describe()

,Delta_t,Tiempo_acumulado,#Autos
count,657.000000,657.000000,657.000000
mean,5.491158,1743.427922,330.000000
std,6.089674,1016.857424,189.803846
min,-0.105144,4.372750,2.000000
25%,2.071410,912.803000,166.000000
50%,3.638480,1655.580000,330.000000
75%,6.292270,2585.910000,494.000000
max,51.434700,3607.690000,658.000000


In [ ]:
sns.distplot(data["T"], hist=False, kde=True, 
             color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

In [ ]:
data[(data["T"] >= -4) & (data["T"] <= 4)]["T"].hist(bins = 1000)